# Administration: Elevate User to Tester

## Config: Endpoints and User Mail Address

### Static configuration variables

In [ ]:
userMail = "kaiser@itis.swiss"

## Clone oSparc Repo and install python dependencies

In [ ]:
print("FYI: You can ignore even fatal errors in this section....")
!git clone https://github.com/ITISFoundation/osparc-simcore.git
!python -m pip install sqlalchemy
!python -m pip install psycopg2-binary

## Fetching Information

### Fetching Information: pgSQL Database

In [ ]:
import sqlalchemy as db
import psycopg2
import os
import json
#######################
osparcURL = "https://" + os.environ['MACHINE_FQDN']
PG_PASSWORD = os.environ.get('POSTGRES_PASSWORD')
PG_ENDPOINT=os.environ.get('POSTGRES_ENDPOINT')
PG_DB=os.environ.get('POSTGRES_DB')
PG_USER=os.environ.get('POSTGRES_USER')
pgEngineURL= "postgresql://{user}:{password}@{host}:{port}/{database}".format(
        user=PG_USER,
        password=PG_PASSWORD,
        database=PG_DB,
        host=PG_ENDPOINT.split(":")[0],
        port=int(PG_ENDPOINT.split(":")[1]),
    )
engine = db.create_engine(pgEngineURL)
connection = engine.connect()
metadata = db.MetaData()
users = db.Table('users', metadata, autoload=True, autoload_with=engine)
##########
# Automatically building datestamp iterator for current invite:
query = db.select([users]).where(users.c.email.like("%" + userMail + "%"))
ResultProxy = connection.execute(query)
ResultSet = ResultProxy.fetchall()

## Modifying Database: Elevate to TESTER

In [ ]:
#Inserting new record into database
query = users.update().\
       values(role="TESTER").\
       where(users.c.email == userMail)
ResultProxy = connection.execute(query)